# Data Exploration and Analysis

## Project Description

[Predict Future Sales](https://www.kaggle.com/c/competitive-data-science-predict-future-sales/) is a Kaggle challenge related to time series. In particular, the proposed task has to do with forecasting the number of products sold in every shop of **1C Company**, a Russian company whose business core revolves around software products. For this purpose, a dataset consisting of daily sales for each (Shop, Item) tuple is provided. These sale records range from January 2013 to October 2015. Instead, the test set corresponds to the monthly amount of items sold in November 2015.

To get a sense of the dataset, an overview of its attributes is presented below:

* **shop_id**: the identifier of a shop
* **shop_name**: the name of a shop
* **item_id**: the identifier of a product
* **item_name**: the name of a product
* **item_category_id**: the identifier of a category. The item category is an "n:1" relationship. To put it in other words, a category is a parent entity that has many related items (or children).
* **item_category_name**: the description of a category
* **item_cnt_day**: the daily selling volume of the product $item\_id$ in the shop $shop\_id$
* **item_price**: price of the product $item\_id$ on $date$. (This is the price for a single unit.)
* **date**: date in format "dd.mm.yyyy"
* **date_block_num**: a consecutive index, such that January 2013 is represented as **0** and October 2015 as **33**.

In total, there are 2,935,849 daily sales records. The test set comprises of 214,200 (Shop, Item) tuples.

## 0. Requirements

In order to encourage reproducibility, the following is a list of technologies used, as well as their respective version:

1. Python **3.7.2**
2. NumPy **1.17.2**
3. SciPy **1.3.1**
4. pandas **0.25.1**
5. Matplotlib **3.1.1**

In [ ]:
import calendar
import json
import os
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display
from IPython.display import Javascript
from IPython.display import HTML

In [ ]:
CURRENT_PATH = os.path.abspath(os.getcwd())
DATA_PATH = os.path.join(CURRENT_PATH, 'data')

In [ ]:
FNAMES = {
    'train': 'sales_train.csv',
    'items': 'items.csv',
    'categories': 'item_categories.csv',
    'shops': 'shops.csv',
    'share': 'market-share.json'
    }

FNAMES = {key: os.path.join(DATA_PATH, fname) for key, fname in FNAMES.items()}

In [ ]:
DATE_FORMAT = '%d-%m-%Y'

MONTH_NAME = {i: abbr for i, abbr in enumerate(calendar.month_abbr) if i > 0}
DAY_NAME = {i: abbr for i, abbr in enumerate(calendar.day_abbr)}

## 1. Data Preprocessing

Let's merge daily sales records with item, category, and shop data to create a more comprehensive reference.

In [ ]:
df = pd.read_csv(FNAMES['train'])

df['date'] = pd.to_datetime(df['date'].str.replace('.', '-', regex='False'),
                            format=DATE_FORMAT)

df = df.astype({
    c: (np.float64 if re.search(r'_(?:day|price)$', c) else np.int64) for c in df.columns if c != 'date'
    })
df = df.rename(columns={'item_cnt_day': 'daily_amount'})

In [ ]:
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['dayofweek'] = df['date'].dt.dayofweek

In [ ]:
categories = pd.read_csv(FNAMES['categories'])

items = pd.read_csv(FNAMES['items'])
items = pd.merge(items, categories, on='item_category_id', how='inner')

shops = pd.read_csv(FNAMES['shops'])

In [ ]:
COLUMNS = [
    'year', 'month', 'date_block_num', 'day', 'dayofweek',
    'shop_id', 'shop_name',
    'item_id', 'item_name', 'item_category_id', 'item_category_name',
    'item_price', 'daily_amount']

df = pd.merge(df, items, on='item_id', how='inner')
df = pd.merge(df, shops, on='shop_id', how='inner')

df = df[COLUMNS]

In [ ]:
sales = df.copy(deep=True)

sales.loc[(sales['daily_amount'] < 0), 'daily_amount'] = 0
sales.loc[(sales['item_price'] < 0), 'item_price'] = 0
    
sales['daily_sales'] = sales['daily_amount'] * sales['item_price']

## 2. Market Share

Let's see how each item category accounts for a share of the total sales.

In [ ]:
display(Javascript('''require.config({paths: {google: 'https://www.gstatic.com/charts/loader'}});'''))
display(Javascript(filename='share.js'))
display(HTML(filename="share.css.html"))

In [ ]:
def draw_market_share(data, width=960, height=400):
    """Draw a chart representing the market share.
    
    Please note that this chart is a Javascript visualization.
    
    Parameters
    ----------
    data : list
        A Python list of lists. Find more regarding its structure at <https://developers.google.com/chart/interactive/docs/gallery/treemap>.
        
    Source: <https://www.stefaanlippens.net/jupyter-custom-d3-visualization.html>
    """
    display(
        Javascript('''
            (function(element) {{
                require(['share'], function(share) {{
                    share(element.get(0), {}, {}, {})
                }});
            }})(element);
            '''.format(json.dumps(data), width, height)
        )
    )

In [ ]:
def market_share(sales, items_per_category=10):
    """Compute data for drawing a market share chart.
    
    The final data structure is highly coupled to the Google Charts library.
    
    Parameters
    ----------
    sales : pd.DataFrame
    """
    data = [['Id', 'Parent', 'Total Sales']]
    
    ancestor = '1C Company'
    data.append([ancestor, None, 0])
    
    data += [
        [category, ancestor, None] for category in sales['item_category_name'].drop_duplicates().tolist()
        ]
    
    items = sales.groupby(['item_category_name', 'item_name'])['daily_sales'].sum().reset_index()
    items = items[['item_name', 'item_category_name', 'daily_sales']]
    items['daily_sales'] = items['daily_sales'].astype(np.int64)
    
    items = items.sort_values(
        by=['item_category_name', 'daily_sales', 'item_name'],
        ascending=[True, False, True]
        )
    
    cat, count = None, 0
    for item, category, sales in items.values.tolist():
        if cat is None or cat != category:
            cat, count = category, 0
        
        if count < (items_per_category - 1):
            data.append([item, category, sales])
        elif count == (items_per_category - 1):
            data.append(['(Others: {})'.format(category), category, sales])
        else:
            data[-1][2] += sales
        
        count += 1
    
    return data

In [ ]:
if not os.path.isfile(FNAMES['share']):
    data = market_share(sales.copy(deep=True))
    
    with open(FNAMES['share'], 'w') as f:
        json.dump(data, f)

In [ ]:
with open(FNAMES['share']) as f:
    data = json.load(f)

draw_market_share(data, 968)

As depicted in the above chart, selling games for the PS3 console, i.e., the "Игры - PS3" item category, corresponds to the largest source of revenue (12.13%). However, as the launch date of the PlayStation 4 is within the range of the daily sale records provided, the sales volume of this console, i.e., the "Игровые консоли - PS4" item category, is approximately four times larger than that of the PlayStation 3 (2.39%). Therefore, the sale of games for PS4 is expected to become the main source of revenue soon.

In a like manner, there are also other interesting findings to highlight, namely:

* The sales volume of games for the eighth-generation console of the PlayStation franchise is nine times larger than that of the Xbox franchise.
* Concerning the sales of eighth-generation consoles, the PlayStation franchise sells approximately six times more than the Xbox franchise.
* In view of the above, we unequivocally conclude that by October 2015 the eighth-generation console race was led by the PlayStation franchise.
* One in three Russian Rubles, apparently the currency of the sale records provided, comes from selling items related to seventh- and eighth-generation consoles of the PlayStation franchise, both games as well as consoles themself.

Lastly, please note that a left-click is required to zoom in the visualization, i.e., analyzing an item category. Instead, a right-click zooms out the visualization, i.e., getting back the main view.

Moreover, let's see how every shop contributes to the total revenue and the sales volume.

In [ ]:
shops = sales.groupby(['shop_name']).agg({'daily_sales': 'sum', 'daily_amount': 'sum'}).reset_index()\
        .rename(columns={'daily_sales': 'revenue', 'daily_amount': 'sales_volume'})

shops['revenue'] /= shops['revenue'].sum()
shops['sales_volume'] /= shops['sales_volume'].sum()

X = shops.melt(id_vars='shop_name', var_name='aggregation', value_name='value')\
    .pivot(index='shop_name', columns='aggregation', values='value')

fig, ax = plt.subplots(figsize=(18, 9))

X.plot(kind='bar', ax=ax)

ax.grid(False)
ax.set_ylabel('Fraction')
ax.set_xlabel('Shop')

plt.show()

Overall, every shop contributes the same to the total revenue and the sales volume. Particularly, the *Москва ТЦ "Семеновский"* shop contributes the most to both aggregation metrics, i.e., 6.9% and 8.5% respectively, whereas *Новосибирск ТРЦ "Галерея Новосибирск"* seems to be a recently opened shop as its contribution is marginal.

## 3. Seasonality

Let's see how revenue generation and sales volume vary depending on the months of the year.

In [ ]:
seasonality = sales.copy(deep=True)

fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(18, 12))

columns = [['daily_sales', 'Revenue Generation'], ['daily_amount', 'Sales Volume']]
projection = [['month', MONTH_NAME], ['dayofweek', DAY_NAME]]

for i, (attr, title) in enumerate(columns):
    
    for year in seasonality['year'].unique():
        mask = (seasonality['year'] == year)
        seasonality.loc[mask, attr] /= seasonality.loc[mask][attr].sum()
    
    for j, (level, mapping) in enumerate(projection):
        X = seasonality.groupby(['year', level])[attr].sum().reset_index()\
            .pivot(index=level, columns='year', values=attr)
        
        X.plot(kind='bar', ax=ax[i, j])
        
        ax[i, j].grid(True)
        
        ax[i, j].set_ylim([0, 0.21])
        ax[i, j].set_ylabel('Fraction')
        
        ax[i, j].set_xticklabels([mapping[int(item.get_text())] for item in ax[i, j].get_xticklabels()])        
        ax[i, j].set_xlabel(None)
        
        ax[i, j].set_title(title)

plt.show()